#### Extracting Time-Series Features and Preprocessing Clinical Notes

In [1]:
# import libraries
import re
import os
import pandas as pd
import numpy as np
import nltk
from nltk import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
## Helper functions for preprocessing text in the data file
# This code is taken from https://github.com/kaggarwal/ClinicalNotesICU

SECTION_TITLES = re.compile(
    r'('
    r'ABDOMEN AND PELVIS|CLINICAL HISTORY|CLINICAL INDICATION|COMPARISON|COMPARISON STUDY DATE'
    r'|EXAM|EXAMINATION|FINDINGS|HISTORY|IMPRESSION|INDICATION'
    r'|MEDICAL CONDITION|PROCEDURE|REASON FOR EXAM|REASON FOR STUDY|REASON FOR THIS EXAMINATION'
    r'|TECHNIQUE'
    r'):|FINAL REPORT',
    re.I | re.M)


def getSentences(t):
    return list(preprocess_mimic(t))

def pattern_repl(matchobj):
    """
    Return a replacement string to be used for match object
    """
    return ' '.rjust(len(matchobj.group(0)))

def clean_text(text):
    """
    Clean text
    """

    # Replace [**Patterns**] with spaces.
    text = re.sub(r'\[\*\*.*?\*\*\]', pattern_repl, text)
    # Replace `_` with spaces.
    text = re.sub(r'_', ' ', text)

    start = 0
    end = find_end(text)
    new_text = ''
    if start > 0:
        new_text += ' ' * start
    new_text = text[start:end]

    # make sure the new text has the same length of old text.
    if len(text) - end > 0:
        new_text += ' ' * (len(text) - end)
    return new_text

def preprocess_mimic(text):
    """
    Preprocess reports in MIMIC-III.
    1. remove [**Patterns**] and signature
    2. split the report into sections
    3. tokenize sentences and words
    4. lowercase
    """
    for sec in split_heading(clean_text(text)):
        for sent in sent_tokenize(sec):
            text = ' '.join(word_tokenize(sent))
            yield text.lower()

def split_heading(text):
    """Split the report into sections"""
    start = 0
    for matcher in SECTION_TITLES.finditer(text):
        # add last
        end = matcher.start()
        if end != start:
            section = text[start:end].strip()
            if section:
                yield section

        # add title
        start = end
        end = matcher.end()
        if end != start:
            section = text[start:end].strip()
            if section:
                yield section

        start = end

    # add last piece
    end = len(text)
    if start < end:
        section = text[start:end].strip()
        if section:
            yield section

def find_end(text):
    """Find the end of the report."""
    ends = [len(text)]
    patterns = [
        re.compile(r'BY ELECTRONICALLY SIGNING THIS REPORT', re.I),
        re.compile(r'\n {3,}DR.', re.I),
        re.compile(r'[ ]{1,}RADLINE ', re.I),
        re.compile(r'.*electronically signed on', re.I),
        re.compile(r'M\[0KM\[0KM')
    ]
    for pattern in patterns:
        matchobj = pattern.search(text)
        if matchobj:
            ends.append(matchobj.start())
    return min(ends)

In [4]:
DATAPATH = "data/"

In [ ]:
# load MIMIC III raw data
def load_raw_data(raw_data_dir):
  """Loads raw data to dataframe/numpy array/tensor"""
  admission_df = pd.read_csv(os.path.join(raw_data_dir, 'ADMISSIONS.csv'))
  events_df = pd.read_csv(os.path.join(raw_data_dir, 'NOTEEVENTS.csv'), low_memory = False)
  icu_stay_df = pd.read_csv(os.path.join(raw_data_dir, 'ICUSTAYS.csv'))

  #print(len(admission_df))
  #print(len(events_df))
  #print(len(icu_stay_df))

  return [admission_df, events_df, icu_stay_df]

raw_data = load_raw_data(DATAPATH)

In [3]:
# Calculate statistics of MIMIC-III data

def calculate_stats(raw_data):
  """Implement this function to calculate the statistics.
  It is encouraged to print out the results."""
  admission_data = raw_data[0]
  note_events_data = raw_data[1]
  icu_stay_data = raw_data[2]
  print("There are a total of ", len(admission_data), " Admission Data")
  print("There are a total of ", len(icu_stay_data), " ICU Stay Data")

  ## Select Clinical Notes
  note_events_data.groupby(note_events_data.CATEGORY).agg(['count'])
  note_categories = note_events_data.groupby(note_events_data.CATEGORY).agg(['count']).index
  selected_note = []
  for category in list(note_categories):
    if category != 'Discharge summary':
      selected_note.append(category)

  ## Create sub notes based on category
  sub_notes = note_events_data[note_events_data.CATEGORY.isin(selected_note)]
  sub_notes.shape

  ## Handle missing chart
  missing_chart_notes_idx = []
  for note in sub_notes.itertuples():
    if isinstance(note.CHARTTIME, str):
      continue
    if np.isnan(note.CHARTTIME):
      missing_chart_notes_idx.append(note.Index)
  print("{} of notes missing charttime.".format(len(missing_chart_notes_idx)))
  sub_notes.drop(missing_chart_notes_idx, inplace = True)
  print("sub_notes shape: ", sub_notes.shape)

  ## Select based on Time Limit (24 hours)
  MIMIC_EXTRACT_DATA = os.path.join(DATAPATH, 'all_hourly_data.h5')
  statistic = pd.read_hdf(MIMIC_EXTRACT_DATA, 'patients')
  print("MIMIC-EXTRACT DATA (Patients Num & Hospital Admission & ICU Admission): ", len(statistic))
  TIMELIMIT = 1 ## 1 day
  statistic.shape
  statistic.head()
  new_stats = statistic.reset_index()
  new_stats.rename(columns = {"subject_id": "SUBJECT_ID", "hadm_id": "HADM_ID"}, inplace = True)
  print("new_stats shape: ", new_stats.shape, "\nsub_notes shape: ", sub_notes.shape)
  df_adm_notes = pd.merge(sub_notes[['ROW_ID','SUBJECT_ID','HADM_ID','CHARTTIME', 'CATEGORY', 'TEXT']],
                          new_stats[['SUBJECT_ID','HADM_ID','icustay_id','age','admittime','dischtime', 'deathtime', 'intime', 'outtime', 'los_icu', 'mort_icu', 'mort_hosp', 'hospital_expire_flag', 'hospstay_seq', 'max_hours']],
                          on = ['SUBJECT_ID'],
                          how = 'left')
  df_adm_notes.head()
  df_adm_notes['CHARTTIME'] = pd.to_datetime(df_adm_notes['CHARTTIME'])
  df_less_n = df_adm_notes[((df_adm_notes['CHARTTIME'] - df_adm_notes['intime']).dt.total_seconds() / (24*60*60)) < TIMELIMIT]
  print("df_less_n.shape: ", df_less_n.shape)

  # Save clinical notes
  pd.to_pickle(df_less_n, os.path.join(DATAPATH, 'sub_notes.p'))
  return df_less_n

state = calculate_stats(raw_data)

NameError: name 'raw_data' is not defined

In [ ]:
# Process raw data

def process_data(raw_data):
  """Preprocess the text in the clinical notes"""
  clinical_notes = pd.read_pickle(os.path.join(DATAPATH, 'sub_notes.p'))
  print("Clinical notes shape: ", clinical_notes.shape)

  eliminate_notes = clinical_notes[clinical_notes.SUBJECT_ID.notnull()]
  eliminate_notes = eliminate_notes[eliminate_notes.CHARTTIME.notnull()]
  eliminate_notes = eliminate_notes[eliminate_notes.TEXT.notnull()]

  eliminate_notes = eliminate_notes[['SUBJECT_ID', 'HADM_ID_y', 'CHARTTIME', 'TEXT']]
  eliminate_notes['preprocessed_text'] = None

  for notes in eliminate_notes.itertuples():
    text = notes.TEXT
    eliminate_notes.at[notes.Index, 'preprocessed_text'] = getSentences(text)

  pd.to_pickle(eliminate_notes, os.path.join(DATAPATH, 'preprocessed_notes.p'))
  print("Preprocessed Data shape: ", eliminate_notes.shape)
  return eliminate_notes

processed_data = process_data(raw_data)


Clinical notes shape:  (252576, 20)
Preprocessed Data shape:  (252576, 5)


In [ ]:
processed_data.columns

Index(['SUBJECT_ID', 'HADM_ID_y', 'CHARTTIME', 'TEXT', 'preprocessed_text'], dtype='object')

In [ ]:
processed_data.head()

,SUBJECT_ID,HADM_ID_y,CHARTTIME,TEXT,preprocessed_text
25,27866,109679.0,2143-04-09 03:22:00,Ms. [**Known lastname 1170**] is an 83 yo F w/...,"[ms. is an 83 yo f w/pmhx sx for cll , hyperte..."
26,27866,109679.0,2143-04-09 03:22:00,Ms. [**Known lastname 1170**] is an 83 yo F w/...,"[ms. is an 83 yo f w/pmhx sx for cll , hyperte..."
39,31975,165003.0,2138-02-11 05:09:00,"Pneumonia, other\n Assessment:\n Afebrile,...","[pneumonia , other assessment : afebrile , wit..."
45,29075,179159.0,2116-02-06 00:25:00,Chief Complaint: GI bleeding\n HPI:\n 67M...,[chief complaint : gi bleeding hpi : 67m w/ h/...
46,29075,179159.0,2116-02-06 00:25:00,Chief Complaint: GI bleeding\n HPI:\n 67M...,[chief complaint : gi bleeding hpi : 67m w/ h/...
